In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score

Загрузка данных и считывание основной информации

In [5]:
data = pd.read_csv('C:/Users/Nastya/Desktop/data_analyse/DATA-ANALYSYS_2-master/Dataset/ORGANICS.csv', encoding="cp1251")
print(data.shape)
print(data.info())

(22223, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 13 columns):
ID                 22223 non-null int64
DemAffl            21138 non-null float64
DemAge             20715 non-null float64
DemCluster         21549 non-null float64
DemClusterGroup    21549 non-null object
DemGender          19711 non-null object
DemReg             21758 non-null object
DemTVReg           21758 non-null object
PromClass          22223 non-null object
PromSpend          22223 non-null float64
PromTime           21942 non-null float64
TargetBuy          22223 non-null int64
TargetAmt          22223 non-null int64
dtypes: float64(5), int64(3), object(5)
memory usage: 2.2+ MB
None


Обработка пропущенных значений

In [6]:
data = data.fillna(data.mean())

Выбор количественных столбцов

In [7]:
x=data[['DemAffl',  'DemAge', 'DemCluster', 'PromSpend', 'PromTime']]
y=data.values[::,11]
y=y.astype('int')

Обучение решающего дерева и его анализ

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 1)
tree=DecisionTreeClassifier(max_depth=5, random_state=1, min_samples_leaf = 1)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best')

In [38]:
print("Точность на обуч наборе", tree.score(X_train, y_train))
print("Точность на тестовом наборе", tree.score(X_test, y_test))

Точность на обуч наборе 0.8034439311213776
Точность на тестовом наборе 0.8052555795536357


In [41]:
from sklearn.metrics import accuracy_score
y_pred_train = tree.predict(X_train)
print("Доля правильных ответов на обуч выборке", accuracy_score(y_pred_train, y_train))

Доля правильных ответов на обуч выборке 0.8034439311213776


In [42]:
y_pred_test = tree.predict(X_test)
print("Доля правильных ответов на тест выборке", accuracy_score(y_pred_test, y_test))

Доля правильных ответов на тест выборке 0.8052555795536357


Разделение выборки на блоки. Кросс-валидация

In [43]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [44]:
print("Значения правильности перекрестной проверки:\n{}".format(
      cross_val_score(tree, x, y, cv=kfold)))

Значения правильности перекрестной проверки:
[0.79932508 0.8071991  0.80292463 0.7929793  0.80153015]


In [45]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("Значения правильности перекрестной проверки:\n{}".format(
    cross_val_score(tree, x, y, cv=kfold)))

Значения правильности перекрестной проверки:
[0.80183585 0.80359071 0.79424868]


Поиск оптимальных параметров для модели

In [47]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(DecisionTreeClassifier(random_state=13),
                  param_grid = {'max_depth' : [3, 4, 5, 6, 7, 8, 9, 10, None],
                                'max_features' : ['auto', 'log2', None],
                                'min_samples_leaf' : range(1, 10),
                                'min_samples_split' : range(2, 10),
                                'criterion' : ['gini', 'entropy']},
                  cv=5, scoring='accuracy'
                  ).fit(X_train, y_train)

In [50]:
print("Оптимальные параметры модели:\n{}".format(gs.best_params_))

Оптимальные параметры модели:
{'criterion': 'gini', 'max_depth': 4, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [51]:
print("Лучшее усредненное значение доли правильных ответов: {:.2f}".format(gs.best_score_))

Лучшее усредненное значение доли правильных ответов: 0.80
